Initialization

In [8]:
# Loading some modules
%load_ext autoreload
%autoreload 2

# Getting basic libraries:
import os, sys, math
import numpy as np
from Bio.PDB import *

# Libraries for structure checking: --> 
import biobb_structure_checking
import biobb_structure_checking.constants as cts
from biobb_structure_checking.structure_checking import StructureChecking
base_dir_path=biobb_structure_checking.__path__[0]
args = cts.set_defaults(base_dir_path,{'notebook':True})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'biobb_structure_checking'

PREPARATION

In [3]:
# 1 - Obtain the required structure from the PDB
parser = PDBParser()
structure = parser.get_structure("6m0j","6m0j.pdb")

/home/jaume/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 6961.
  warnings.warn(
/home/jaume/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 7019.
  warnings.warn(
/home/jaume/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 7033.
  warnings.warn(
/home/jaume/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 7104.
  warnings.warn(


In [ ]:
# 2 - Determine the contribution to the stability of the complex by mimicking a traditional Ala-scanning experiment, i.e.
# replacing each residue in turn by Ala and evaluating the changes in the complex interaction energy


In [ ]:
# 3 - Remove all heteroatoms


In [ ]:
# 4 - Perform a quality checking on the structures, and add missing side-chains and hydrogen atoms and atom charges, using
# the biobb_structure_checking module